# Householder Similarity Transforms

Copyright (C) 2020 Andreas Kloeckner

<details>
<summary>MIT License</summary>
Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in
all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
THE SOFTWARE.
</details>

In [1]:
import numpy as np
import numpy.linalg as la

np.set_printoptions(precision=2, linewidth=150)

In [2]:
n = 8

e1 = np.zeros(n); e1[0] = 1
e2 = np.zeros(n); e2[1] = 1

A = np.random.randn(n, n)
A

array([[ 0.93,  1.94,  1.6 , -0.61,  1.01, -0.19, -0.93,  1.03],
       [ 0.99, -1.25,  1.11, -0.29, -0.74, -1.35, -0.33, -0.37],
       [-1.36, -0.25,  0.26,  0.48,  1.24, -1.56,  0.43, -0.11],
       [-1.42,  0.46, -0.04,  0.65,  0.34,  2.59,  0.95,  1.34],
       [ 0.42, -0.4 ,  2.32, -0.38, -0.26, -0.76,  1.24,  0.15],
       [ 0.01,  0.98,  0.95, -0.1 ,  0.37, -0.71,  2.11,  0.22],
       [ 0.3 , -0.21,  0.88, -0.77, -1.13,  0.46, -0.78, -1.39],
       [ 0.16, -1.45, -0.57, -0.42,  1.51,  0.5 ,  1.44,  0.59]])

Now try to zero the first column with a similarity transform.

## Starting with the first row

Lets first try to proceed as in Householder QR, creating a transformation
$$H=I-2\frac{vv^T}{v^Tv}$$
where $v = a_1 - ||a_1||_2e_1$ with $a_1$ being the first column of $A$.

In [3]:
#clear
a = A[:, 0].copy()
v = a-la.norm(a)*e1

H1 = np.eye(n) - 2*np.outer(v, v)/(v@v)

We can apply the transformation from the left as in QR to reduce the first column to a multiple of the first elementary vector.

In [4]:
#clear
(H1@A).round(4)

array([[ 2.45, -0.08,  1.4 , -1.18, -0.89, -1.29, -1.06, -0.58],
       [-0.  ,  0.07,  1.24,  0.08,  0.49, -0.63, -0.25,  0.67],
       [ 0.  , -2.06,  0.09, -0.03, -0.46, -2.55,  0.32, -1.55],
       [ 0.  , -1.42, -0.22,  0.12, -1.43,  1.56,  0.83, -0.15],
       [-0.  ,  0.16,  2.37, -0.22,  0.26, -0.46,  1.28,  0.59],
       [-0.  ,  0.99,  0.95, -0.1 ,  0.39, -0.7 ,  2.11,  0.24],
       [-0.  ,  0.18,  0.91, -0.66, -0.75,  0.68, -0.76, -1.07],
       [-0.  , -1.24, -0.55, -0.36,  1.7 ,  0.61,  1.45,  0.75]])

However, to ensure we do not perturb the eigenvalues of $A$, we must also apply the matrix from the right, resulting in a similarity transformation.

In [5]:
(H1@A@H1.T).round(4)

array([[ 0.47,  1.21, -0.37, -3.02, -0.34, -1.27, -0.67, -0.38],
       [-0.61,  0.47,  0.69, -0.5 ,  0.66, -0.63, -0.13,  0.74],
       [-1.01, -1.4 , -0.82, -0.97, -0.18, -2.54,  0.52, -1.45],
       [-0.66, -0.99, -0.81, -0.5 , -1.25,  1.57,  0.96, -0.08],
       [-0.9 ,  0.74,  1.57, -1.05,  0.51, -0.45,  1.45,  0.69],
       [ 0.26,  0.82,  1.19,  0.15,  0.31, -0.7 ,  2.06,  0.21],
       [-0.34,  0.4 ,  0.61, -0.98, -0.66,  0.68, -0.69, -1.04],
       [ 0.54, -1.59, -0.07,  0.14,  1.56,  0.61,  1.34,  0.7 ]])

Note that applying the Householder transformation from the right filled in the elements annihilated by applying it from the left.

## Starting in the second row

To avoid this, we define the Householder transformation to annihilate elements below the first subdiagonal. That way, the first transformation does not affect the first row when applied from the left, and consequently does not affect the first column when applied for the right, preserving the zeros we've annihilated.

In [6]:
#clear
a = A[:, 0].copy()
a[0] = 0
v = a-la.norm(a)*e2

H2 = np.eye(n) - 2*np.outer(v, v)/(v@v)

In [7]:
#clear
(H2 @ A).round(4)

array([[ 0.93,  1.94,  1.6 , -0.61,  1.01, -0.19, -0.93,  1.03],
       [ 2.27, -0.88,  0.86, -1.02, -1.38, -1.32, -0.76, -1.05],
       [ 0.  ,  0.15, -0.  , -0.3 ,  0.57, -1.52, -0.02, -0.84],
       [ 0.  ,  0.87, -0.32, -0.16, -0.36,  2.62,  0.47,  0.59],
       [-0.  , -0.52,  2.4 , -0.14, -0.06, -0.77,  1.38,  0.37],
       [-0.  ,  0.97,  0.95, -0.09,  0.38, -0.71,  2.11,  0.23],
       [-0.  , -0.3 ,  0.93, -0.6 , -0.98,  0.45, -0.68, -1.23],
       [-0.  , -1.49, -0.54, -0.33,  1.59,  0.5 ,  1.49,  0.67]])

In [8]:
#clear
(H2 @ A @ H2.T).round(4)

array([[ 0.93,  0.4 , -0.05, -2.32,  1.52, -0.17, -0.57,  1.21],
       [ 2.27, -0.7 ,  1.06, -0.82, -1.44, -1.32, -0.8 , -1.07],
       [ 0.  ,  0.29,  0.15, -0.14,  0.52, -1.53, -0.06, -0.85],
       [ 0.  ,  0.72, -0.48, -0.33, -0.31,  2.62,  0.51,  0.61],
       [-0.  , -1.39,  1.47, -1.11,  0.23, -0.77,  1.59,  0.48],
       [-0.  ,  0.27,  0.2 , -0.87,  0.61, -0.7 ,  2.28,  0.32],
       [-0.  , -0.67,  0.54, -1.01, -0.86,  0.46, -0.6 , -1.19],
       [-0.  ,  0.42,  1.5 ,  1.79,  0.96,  0.48,  1.04,  0.44]])

To generalize this process, we continue to eliminate everything below the subdiagonal in the next column and applying the two-sided transformations, finally resulting in an upper-Hessenberg matrix.

-----

Why does post-multiplying with `H2.T` not destroy the zeros?

In [9]:
H2.T.round(4)

array([[ 1.00e+00,  0.00e+00,  0.00e+00,  0.00e+00,  0.00e+00,  0.00e+00,  0.00e+00,  0.00e+00],
       [ 0.00e+00,  4.36e-01, -6.02e-01, -6.25e-01,  1.85e-01,  5.40e-03,  1.32e-01,  6.85e-02],
       [ 0.00e+00, -6.02e-01,  3.57e-01, -6.68e-01,  1.98e-01,  5.70e-03,  1.41e-01,  7.32e-02],
       [ 0.00e+00, -6.25e-01, -6.68e-01,  3.07e-01,  2.05e-01,  5.90e-03,  1.47e-01,  7.59e-02],
       [ 0.00e+00,  1.85e-01,  1.98e-01,  2.05e-01,  9.39e-01, -1.80e-03, -4.34e-02, -2.25e-02],
       [ 0.00e+00,  5.40e-03,  5.70e-03,  5.90e-03, -1.80e-03,  1.00e+00, -1.30e-03, -7.00e-04],
       [ 0.00e+00,  1.32e-01,  1.41e-01,  1.47e-01, -4.34e-02, -1.30e-03,  9.69e-01, -1.61e-02],
       [ 0.00e+00,  6.85e-02,  7.32e-02,  7.59e-02, -2.25e-02, -7.00e-04, -1.61e-02,  9.92e-01]])